# Importing important libraries

In [1]:
import pyspark
from pyspark.sql import SparkSession #connecting session

from pyspark.sql.functions import *
from pyspark.sql.functions import isnan, isnull , when, count  #identifying number of null values
from pyspark.sql.functions import trim  #remove extra spaces from statement
from pyspark.sql.functions import lower # bring all string in lower case
#from pyspark.sql.functions import format_number ## same like format and rounding of in pandas

from pyspark.sql.types import IntegerType, DoubleType,StringType # changing datatype of the variable

from pyspark.ml.feature import StringIndexer # converting string category into numbers ( same like label encoder)
from pyspark.ml.feature import VectorAssembler # selecting all independent variable and making a list

from pyspark.ml.classification import GBTClassifier # xgb classifier
from pyspark.ml.classification import RandomForestClassifier # random forest classifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator # model evaluation library
##from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Spark session building

In [2]:
spark = SparkSession.builder.appName('modelbuilding').getOrCreate()

# Reading dataset (importing)

In [3]:
# df = spark.read.option("header", "true") \
#     .option("delimiter", "|") \
#     .option("inferSchema", "true") \
#     .csv('G:\\HFL_base_other_investment_targets.csv')
#orrrrr

df = spark.read.csv('G:\\data.csv', inferSchema=True, header=True)

In [4]:
df.show(1) #, truncate=False)

+------+------+------+-------------+--------------+-----------------------+----------------------+---------------------+----------------+-----------------+--------------+-------------------------+-----------------+-------+-------+--------+----+------+----+
|   tag|gender|tenure|interest_rate|sanctionamount|first_disbursement_date|last_Disbursement_date|sum_Disubursement_amt|overdue_interest|overdue_principal|bounce_charges|principal_Outstanding_amt|no_of_loan_counts|vintage|recency|  Status| Age|Target|uniq|
+------+------+------+-------------+--------------+-----------------------+----------------------+---------------------+----------------+-----------------+--------------+-------------------------+-----------------+-------+-------+--------+----+------+----+
|online|  NULL|  37.0|         23.0|      100000.0|              4/27/2017|              5/5/2020|             100073.0|             0.0|              0.0|           0.0|                      0.0|              1.0|    5.0|    2.0

# checking variables and their types

In [5]:
df.printSchema()

root
 |-- tag: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- sanctionamount: double (nullable = true)
 |-- first_disbursement_date: string (nullable = true)
 |-- last_Disbursement_date: string (nullable = true)
 |-- sum_Disubursement_amt: double (nullable = true)
 |-- overdue_interest: double (nullable = true)
 |-- overdue_principal: double (nullable = true)
 |-- bounce_charges: double (nullable = true)
 |-- principal_Outstanding_amt: double (nullable = true)
 |-- no_of_loan_counts: double (nullable = true)
 |-- vintage: double (nullable = true)
 |-- recency: double (nullable = true)
 |-- Status: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Target: integer (nullable = true)
 |-- uniq: integer (nullable = true)



# fetching number of rows and column

In [7]:
print('total records : ' ,df.count()) #total rows
print('total distinct records : ',df.distinct().count()) #total distinct rows
print ('number of columns : ', len(df.columns))
print( 'distinct number of customers : ',df.select(['uniq']).distinct().count()) 
#total distinct customer since it is the uniqueness

total records :  457890
total distinct records :  457890
number of columns :  19
distinct number of customers :  457890


# column names

In [8]:
df.columns

['tag',
 'gender',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'first_disbursement_date',
 'last_Disbursement_date',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Status',
 'Age',
 'Target',
 'uniq']

# seeing data head

In [9]:
df.toPandas().head(3)

,tag,gender,tenure,interest_rate,sanctionamount,first_disbursement_date,last_Disbursement_date,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Status,Age,Target,uniq
0,online,None,37.0,23.0,100000.0,4/27/2017,5/5/2020,100073.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,INACTIVE,42.0,0,0
1,online,M,6.0,0.0,10400.0,7/7/2017,7/7/2017,10400.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,INACTIVE,55.0,0,1
2,online,M,6.0,36.0,15000.0,8/17/2022,8/17/2022,15000.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,INACTIVE,36.0,0,2


# trim & lower case function

In [10]:
df = df.withColumn("gender", trim(lower(df["gender"])))
df = df.withColumn("tag", trim(lower(df["tag"])))
df = df.withColumn("Status", trim(lower(df["Status"])))

# getting distinct number of categories of variables

In [11]:
print(df.select('tag').distinct().count())
print(df.select('gender').distinct().count())
print(df.select('Status').distinct().count())

2
7
3


# getting distinct name of categories of variables

In [12]:
print(df.select('tag').distinct().show())
print(df.select('gender').distinct().show())
print(df.select('Status').distinct().show())

+------+
|   tag|
+------+
|online|
|offlin|
+------+

None
+------+
|gender|
+------+
|     m|
|     f|
| other|
|others|
|female|
|  male|
|  NULL|
+------+

None
+--------+
|  Status|
+--------+
|  active|
|inactive|
|    NULL|
+--------+

None


# getting distinct category names and their counts

In [13]:
df.groupBy('gender').count().show()

+------+------+
|gender| count|
+------+------+
|     m|301962|
|  NULL| 61903|
|     f| 45179|
| other|     3|
|others|     1|
|female|  7887|
|  male| 40955|
+------+------+



In [14]:
df.groupBy('tag').count().show()

+------+------+
|   tag| count|
+------+------+
|online|356204|
|offlin|101686|
+------+------+



In [15]:
df.groupBy('Status').count().show()

#df.select('Status').distinct().show()

+--------+------+
|  Status| count|
+--------+------+
|    NULL|   156|
|  active|328904|
|inactive|128830|
+--------+------+



# Null values and their percentage

number of missing values

In [16]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

#https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/
# OR

# df.select([count(when(col(c).contains('None') | \
#                             col(c).contains('NULL') | \
#                             (col(c) == '' ) | \
#                             col(c).isNull() | \
#                             isnan(c), c 
#                            )).alias(c)
#                     for c in df.columns]).show(vertical=True)

-RECORD 0--------------------------
 tag                       | 0     
 gender                    | 61903 
 tenure                    | 836   
 interest_rate             | 11    
 sanctionamount            | 16    
 first_disbursement_date   | 2840  
 last_Disbursement_date    | 2840  
 sum_Disubursement_amt     | 0     
 overdue_interest          | 47842 
 overdue_principal         | 47853 
 bounce_charges            | 47853 
 principal_Outstanding_amt | 47062 
 no_of_loan_counts         | 2840  
 vintage                   | 2840  
 recency                   | 2840  
 Status                    | 156   
 Age                       | 390   
 Target                    | 0     
 uniq                      | 0     



In [17]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

,tag,gender,tenure,interest_rate,sanctionamount,first_disbursement_date,last_Disbursement_date,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Status,Age,Target,uniq
0,0,61903,836,11,16,2840,2840,0,47842,47853,47853,47062,2840,2840,2840,156,390,0,0


# % of missing values

In [18]:
df.select([round((count(when(isnan(c)| col(c).isNull(),c))*100/count(lit(1))),6).
           alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0------------------------------
 tag                       | 0.0       
 gender                    | 13.519186 
 tenure                    | 0.182577  
 interest_rate             | 0.002402  
 sanctionamount            | 0.003494  
 first_disbursement_date   | 0.620236  
 last_Disbursement_date    | 0.620236  
 sum_Disubursement_amt     | 0.0       
 overdue_interest          | 10.448361 
 overdue_principal         | 10.450763 
 bounce_charges            | 10.450763 
 principal_Outstanding_amt | 10.278014 
 no_of_loan_counts         | 0.620236  
 vintage                   | 0.620236  
 recency                   | 0.620236  
 Status                    | 0.034069  
 Age                       | 0.085173  
 Target                    | 0.0       
 uniq                      | 0.0       



# fetching selected columns & storing in dataframe

In [21]:
df=df.select(['uniq',
 'tag',
 'gender',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'first_disbursement_date',
 'last_Disbursement_date',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Status',
 'Age',
 'Target'])

In [22]:
len(df.columns)

19

# Dropping columns

# Drop single column

In [23]:
# df= df.drop('Target')
# df=df.drop('uniq')

# Drop multiple columns

In [24]:
df= df.drop(*( 'first_disbursement_date','last_Disbursement_date'))

# variable & datatype

In [25]:
df.dtypes

[('uniq', 'int'),
 ('tag', 'string'),
 ('gender', 'string'),
 ('tenure', 'double'),
 ('interest_rate', 'double'),
 ('sanctionamount', 'double'),
 ('sum_Disubursement_amt', 'double'),
 ('overdue_interest', 'double'),
 ('overdue_principal', 'double'),
 ('bounce_charges', 'double'),
 ('principal_Outstanding_amt', 'double'),
 ('no_of_loan_counts', 'double'),
 ('vintage', 'double'),
 ('recency', 'double'),
 ('Status', 'string'),
 ('Age', 'double'),
 ('Target', 'int')]

# change variable datatype

In [26]:
df=df.withColumn('Age_new',df['Age'].cast(IntegerType()))

In [27]:
df.dtypes

[('uniq', 'int'),
 ('tag', 'string'),
 ('gender', 'string'),
 ('tenure', 'double'),
 ('interest_rate', 'double'),
 ('sanctionamount', 'double'),
 ('sum_Disubursement_amt', 'double'),
 ('overdue_interest', 'double'),
 ('overdue_principal', 'double'),
 ('bounce_charges', 'double'),
 ('principal_Outstanding_amt', 'double'),
 ('no_of_loan_counts', 'double'),
 ('vintage', 'double'),
 ('recency', 'double'),
 ('Status', 'string'),
 ('Age', 'double'),
 ('Target', 'int'),
 ('Age_new', 'int')]

# Renaming columns

In [28]:
df = df.withColumnRenamed('tag', 'tag_cat')
df = df.withColumnRenamed('gender','gender_cat')
df = df.withColumnRenamed('Status','status_cat')

In [29]:
df.columns

['uniq',
 'tag_cat',
 'gender_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'status_cat',
 'Age',
 'Target',
 'Age_new']

In [31]:
df=df.select([
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'Target'])

In [32]:
df.printSchema()

root
 |-- tag_cat: string (nullable = true)
 |-- gender_cat: string (nullable = true)
 |-- status_cat: string (nullable = true)
 |-- tenure: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- sanctionamount: double (nullable = true)
 |-- sum_Disubursement_amt: double (nullable = true)
 |-- overdue_interest: double (nullable = true)
 |-- overdue_principal: double (nullable = true)
 |-- bounce_charges: double (nullable = true)
 |-- principal_Outstanding_amt: double (nullable = true)
 |-- no_of_loan_counts: double (nullable = true)
 |-- vintage: double (nullable = true)
 |-- recency: double (nullable = true)
 |-- Age_new: integer (nullable = true)
 |-- Target: integer (nullable = true)



In [33]:
print('total records : ' ,df.count()) #total rows
print('total distinct records : ',df.distinct().count()) #total distinct rows
print ('number of columns : ', len(df.columns)) 
#total distinct customer since it is the uniqueness

total records :  457890
total distinct records :  396212
number of columns :  16


In [34]:
df.toPandas().head(3)

,tag_cat,gender_cat,status_cat,tenure,interest_rate,sanctionamount,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Age_new,Target
0,online,None,inactive,37.0,23.0,100000.0,100073.0,0.0,0.0,0.0,0.0,1.0,5.0,2.0,42.0,0
1,online,m,inactive,6.0,0.0,10400.0,10400.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,55.0,0
2,online,m,inactive,6.0,36.0,15000.0,15000.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,36.0,0


In [35]:
'''if your categorical feature is already "represented as a label index", 
you don't need to use StringIndexer first. Instead, you can directly apply one-hot encoding.

On the other hand:

if your categorical feature is, e.g. represented as string values, 
it becomes necessary to use StringIndexer first to convert the string values into label indices (numeric values). '''

'if your categorical feature is already "represented as a label index", \nyou don\'t need to use StringIndexer first. Instead, you can directly apply one-hot encoding.\n\nOn the other hand:\n\nif your categorical feature is, e.g. represented as string values, \nit becomes necessary to use StringIndexer first to convert the string values into label indices (numeric values). '

# getting distinct category names and their counts

In [36]:
df.groupBy('gender_cat').count().show()

+----------+------+
|gender_cat| count|
+----------+------+
|         m|301962|
|      NULL| 61903|
|         f| 45179|
|     other|     3|
|    others|     1|
|    female|  7887|
|      male| 40955|
+----------+------+



In [37]:
df.groupBy('tag_cat').count().show()

+-------+------+
|tag_cat| count|
+-------+------+
| online|356204|
| offlin|101686|
+-------+------+



In [38]:
df.groupBy('Status_cat').count().show()

+----------+------+
|Status_cat| count|
+----------+------+
|      NULL|   156|
|    active|328904|
|  inactive|128830|
+----------+------+



# bivariate wrt target variable

In [39]:
df.groupBy('tag_cat').pivot('Target').count().show()

+-------+------+-----+
|tag_cat|     0|    1|
+-------+------+-----+
| online|353885| 2319|
| offlin| 86811|14875|
+-------+------+-----+



In [40]:
df.groupBy('Status_cat').pivot('Target').count().show()

+----------+------+-----+
|Status_cat|     0|    1|
+----------+------+-----+
|      NULL|   149|    7|
|    active|313745|15159|
|  inactive|126802| 2028|
+----------+------+-----+



In [41]:
df.groupBy('gender_cat').pivot('Target').count().show()

+----------+------+-----+
|gender_cat|     0|    1|
+----------+------+-----+
|         m|299982| 1980|
|      NULL| 59160| 2743|
|         f| 44927|  252|
|    female|  6087| 1800|
|     other|     3| NULL|
|    others|     1| NULL|
|      male| 30536|10419|
+----------+------+-----+



# Impute string columns by mode

In [42]:
df = df.fillna({'Status_cat':'active'})
df.groupBy('Status_cat').count().show()

+----------+------+
|Status_cat| count|
+----------+------+
|    active|329060|
|  inactive|128830|
+----------+------+



In [43]:
df=df.na.replace(['male', 'm','f','other','others','female'], ['A','A','B','A','A','B'], 'gender_cat')
#https://www.skytowner.com/explore/pyspark_dataframe_replace_method
#https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.replace.html

In [44]:
df = df.fillna({'gender_cat':'A'})
df.groupBy('gender_cat').count().show()

+----------+------+
|gender_cat| count|
+----------+------+
|         B| 53066|
|         A|404824|
+----------+------+



In [45]:
df.groupBy('Target').count().show()

+------+------+
|Target| count|
+------+------+
|     0|440696|
|     1| 17194|
+------+------+



In [46]:
df.columns

['tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'Target']

In [47]:
df.select([round((count(when(isnan(c)| col(c).isNull(),c))*100/count(lit(1))),6).
           alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0------------------------------
 tag_cat                   | 0.0       
 gender_cat                | 0.0       
 status_cat                | 0.0       
 tenure                    | 0.182577  
 interest_rate             | 0.002402  
 sanctionamount            | 0.003494  
 sum_Disubursement_amt     | 0.0       
 overdue_interest          | 10.448361 
 overdue_principal         | 10.450763 
 bounce_charges            | 10.450763 
 principal_Outstanding_amt | 10.278014 
 no_of_loan_counts         | 0.620236  
 vintage                   | 0.620236  
 recency                   | 0.620236  
 Age_new                   | 0.085173  
 Target                    | 0.0       



In [48]:
len(df.columns)

16

In [49]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0--------------------------
 tag_cat                   | 0     
 gender_cat                | 0     
 status_cat                | 0     
 tenure                    | 836   
 interest_rate             | 11    
 sanctionamount            | 16    
 sum_Disubursement_amt     | 0     
 overdue_interest          | 47842 
 overdue_principal         | 47853 
 bounce_charges            | 47853 
 principal_Outstanding_amt | 47062 
 no_of_loan_counts         | 2840  
 vintage                   | 2840  
 recency                   | 2840  
 Age_new                   | 390   
 Target                    | 0     



# Drop rows having individual row missing value more than 8 out of 16 column

In [50]:
##threshold
df=df.na.drop(how="any",thresh=8)

In [51]:
df.count()  #there are no as such records

457890

In [52]:
df=df.na.drop(how="any",thresh=5)
df.count()

457890

# Dropping rows wrt column having null values

In [53]:
df=df.na.drop(how="any",subset=['overdue_principal'])
df.count()

410037

In [54]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0-------------------------
 tag_cat                   | 0    
 gender_cat                | 0    
 status_cat                | 0    
 tenure                    | 0    
 interest_rate             | 0    
 sanctionamount            | 3    
 sum_Disubursement_amt     | 0    
 overdue_interest          | 0    
 overdue_principal         | 0    
 bounce_charges            | 0    
 principal_Outstanding_amt | 67   
 no_of_loan_counts         | 2768 
 vintage                   | 2768 
 recency                   | 2768 
 Age_new                   | 385  
 Target                    | 0    



In [55]:
df=df.na.drop(how="any",subset=['sanctionamount'])
df=df.na.drop(how="any",subset=['recency'])
df=df.na.drop(how="any",subset=['Age_new'])
df.count()

406881

In [56]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

-RECORD 0------------------------
 tag_cat                   | 0   
 gender_cat                | 0   
 status_cat                | 0   
 tenure                    | 0   
 interest_rate             | 0   
 sanctionamount            | 0   
 sum_Disubursement_amt     | 0   
 overdue_interest          | 0   
 overdue_principal         | 0   
 bounce_charges            | 0   
 principal_Outstanding_amt | 67  
 no_of_loan_counts         | 0   
 vintage                   | 0   
 recency                   | 0   
 Age_new                   | 0   
 Target                    | 0   



In [57]:
df.columns

['tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'Target']

In [58]:
df.printSchema()

root
 |-- tag_cat: string (nullable = true)
 |-- gender_cat: string (nullable = false)
 |-- status_cat: string (nullable = false)
 |-- tenure: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- sanctionamount: double (nullable = true)
 |-- sum_Disubursement_amt: double (nullable = true)
 |-- overdue_interest: double (nullable = true)
 |-- overdue_principal: double (nullable = true)
 |-- bounce_charges: double (nullable = true)
 |-- principal_Outstanding_amt: double (nullable = true)
 |-- no_of_loan_counts: double (nullable = true)
 |-- vintage: double (nullable = true)
 |-- recency: double (nullable = true)
 |-- Age_new: integer (nullable = true)
 |-- Target: integer (nullable = true)



# Data continous var summary

In [59]:
df.select([ 
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
  'overdue_interest',
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-------------+--------------+---------------------+----------------+
|summary|interest_rate|sanctionamount|sum_Disubursement_amt|overdue_interest|
+-------+-------------+--------------+---------------------+----------------+
|  count|       406881|        406881|               406881|          406881|
|    min|          0.0|        1000.0|                  0.0|        -65851.0|
|    25%|         15.0|       29000.0|              39001.0|             0.0|
|    50%|         22.0|       60444.0|             120000.0|             0.0|
|    75%|        27.74|      270000.0|             450000.0|             0.0|
|    97%|         36.0|     2542000.0|            3000000.0|          6928.0|
|    99%|        41.14|     5260000.0|            6837180.0|         33219.0|
|    max|         48.0|         2.5E9|           9.528898E9|   2.381285881E7|
+-------+-------------+--------------+---------------------+----------------+



In [60]:
 df.select([
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-----------------+--------------+-------------------------+-----------------+
|summary|overdue_principal|bounce_charges|principal_Outstanding_amt|no_of_loan_counts|
+-------+-----------------+--------------+-------------------------+-----------------+
|  count|           406881|        406881|                   406814|           406881|
|    min|             -1.0|       -8430.5|                     -1.0|              1.0|
|    25%|              0.0|           0.0|                      0.0|              1.0|
|    50%|              0.0|           0.0|                  25818.0|              1.0|
|    75%|              0.0|           0.0|                 200000.0|              1.0|
|    97%|          20127.0|        7000.0|                2174139.0|              3.0|
|    99%|          59001.0|       23250.0|                4751386.0|              4.0|
|    max|    8.682427012E7|       69108.0|            2.054270434E9|            183.0|
+-------+-----------------+--------------+-

In [61]:
 df.select([
 'tenure',
 'vintage',
 'recency',
 'Age_new'
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+------+-------+-------+-------+
|summary|tenure|vintage|recency|Age_new|
+-------+------+-------+-------+-------+
|  count|406881| 406881| 406881| 406881|
|    min|   1.0|    0.0|    0.0|    -19|
|    25%|   9.0|    0.0|    0.0|     23|
|    50%|  12.0|    0.0|    0.0|     30|
|    75%|  36.0|    1.0|    1.0|     38|
|    97%|  84.0|    4.0|    4.0|     57|
|    99%| 189.0|    5.0|    4.0|     60|
|    max| 727.0|   15.0|    8.0|    947|
+-------+------+-------+-------+-------+



In [62]:
df1=df.select([
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'Target'])

# Drop rows with certain conditions like negative age, outlier record, negative principal and charges

In [63]:
df=df.where(df.tenure<=84.0)
df.count()

397016

In [64]:
df=df.where(df.vintage<=5)
df=df.where(df.recency<=4)
df.count()

393226

In [65]:
df=df.filter((df.Age_new >=21) & (df.Age_new <=60))

In [66]:
df=df.filter((df.principal_Outstanding_amt >=0) & (df.principal_Outstanding_amt <=4751386.0))
df=df.filter((df.no_of_loan_counts >=1) & (df.no_of_loan_counts <=4))
df=df.filter((df.overdue_interest >=0) & (df.overdue_interest <=33219.0))

In [67]:
df.count()

327673

In [68]:
df.select([ 
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
  'overdue_interest',
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-------------+--------------+---------------------+----------------+
|summary|interest_rate|sanctionamount|sum_Disubursement_amt|overdue_interest|
+-------+-------------+--------------+---------------------+----------------+
|  count|       327673|        327673|               327673|          327673|
|    min|          0.0|        2184.0|                  0.7|             0.0|
|    25%|        14.13|       23500.0|              30000.0|             0.0|
|    50%|         22.0|       50000.0|              80000.0|             0.0|
|    75%|         29.0|      109440.0|             220000.0|             0.0|
|    97%|         36.0|      900000.0|            1300000.0|          2711.0|
|    99%|         42.0|     2000000.0|           2727589.21|         10609.0|
|    max|         48.0|  9.09009285E8|           9.528898E9|         33194.0|
+-------+-------------+--------------+---------------------+----------------+



In [69]:
 df.select([
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-----------------+--------------+-------------------------+-----------------+
|summary|overdue_principal|bounce_charges|principal_Outstanding_amt|no_of_loan_counts|
+-------+-----------------+--------------+-------------------------+-----------------+
|  count|           327673|        327673|                   327673|           327673|
|    min|              0.0|       -6500.0|                      0.0|              1.0|
|    25%|              0.0|           0.0|                      0.0|              1.0|
|    50%|              0.0|           0.0|                  15000.0|              1.0|
|    75%|              0.0|           0.0|                  74782.0|              1.0|
|    97%|          14000.0|        3000.0|                 775427.0|              3.0|
|    99%|          34955.0|       19492.0|                1814898.0|              4.0|
|    max|        1000000.0|       69108.0|                4749860.0|              4.0|
+-------+-----------------+--------------+-

In [70]:
 df.select([
 'tenure',
 'vintage',
 'recency',
 'Age_new'
          ]).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+------+-------+-------+-------+
|summary|tenure|vintage|recency|Age_new|
+-------+------+-------+-------+-------+
|  count|327673| 327673| 327673| 327673|
|    min|   1.0|    0.0|    0.0|     21|
|    25%|   8.0|    0.0|    0.0|     27|
|    50%|  12.0|    0.0|    0.0|     32|
|    75%|  23.0|    1.0|    1.0|     39|
|    97%|  61.0|    4.0|    4.0|     56|
|    99%|  84.0|    4.0|    4.0|     58|
|    max|  84.0|    5.0|    4.0|     60|
+-------+------+-------+-------+-------+



# impute variables

In [72]:
# df=df.na.replace([84], [24], 'tenure')
# df=df.na.replace([1000000.0], [36307.0], 'overdue_principal')
# df=df.na.replace([2.05E8], [2500000.0], 'sanctionamount')
# df=df.na.replace([9.752E8], [2500000.0], 'sum_Disubursement_amt')

df=df.where(df.vintage)

# ##this imputation is not as good vo sirf particular value impute karega

In [74]:
df.select([ 
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
  'overdue_interest']).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-------------+--------------+---------------------+----------------+
|summary|interest_rate|sanctionamount|sum_Disubursement_amt|overdue_interest|
+-------+-------------+--------------+---------------------+----------------+
|  count|       214954|        214954|               214954|          214954|
|    min|          0.0|        2189.0|               2335.0|             0.0|
|    25%|         16.0|       25000.0|              47000.0|             0.0|
|    50%|         23.0|       60000.0|             120000.0|             0.0|
|    75%|         30.0|      177000.0|             300000.0|             0.0|
|    97%|         36.0|     1290000.0|            1400000.0|          3774.0|
|    99%|         42.0|     2500000.0|            2500000.0|         11077.0|
|    max|         48.0|        2.05E8|              9.752E8|         33194.0|
+-------+-------------+--------------+---------------------+----------------+



In [75]:
 df.select([
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts']).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+-----------------+--------------+-------------------------+-----------------+
|summary|overdue_principal|bounce_charges|principal_Outstanding_amt|no_of_loan_counts|
+-------+-----------------+--------------+-------------------------+-----------------+
|  count|           214954|        214954|                   214954|           214954|
|    min|              0.0|           0.0|                     23.0|              1.0|
|    25%|              0.0|           0.0|                  15219.0|              1.0|
|    50%|              0.0|           0.0|                  45110.0|              1.0|
|    75%|              0.0|           0.0|                 143577.0|              1.0|
|    97%|          18067.0|        2000.0|                1178369.0|              3.0|
|    99%|          35963.0|       14176.0|                2212753.0|              4.0|
|    max|        1000000.0|       23250.0|                4749860.0|              4.0|
+-------+-----------------+--------------+-

In [76]:
 df.select([
 'tenure',
 'vintage',
 'recency',
 'Age_new']).summary("count",'min','25%','50%','75%','97%','99%','max').show()

+-------+------+-------+-------+-------+
|summary|tenure|vintage|recency|Age_new|
+-------+------+-------+-------+-------+
|  count|214954| 214954| 214954| 214954|
|    min|   1.0|    0.0|    0.0|     20|
|    25%|   9.0|    0.0|    0.0|     26|
|    50%|  12.0|    0.0|    0.0|     32|
|    75%|  24.0|    1.0|    0.0|     38|
|    97%|  84.0|    3.0|    2.0|     56|
|    99%|  84.0|    4.0|    3.0|     58|
|    max|  84.0|    5.0|    4.0|     60|
+-------+------+-------+-------+-------+



In [77]:
#https://deepnote.com/@rajshekar-2021/Outlier-Detection-Pyspark-069e69af-2c1d-4d4d-884a-92aad276d06f

In [78]:
df.printSchema()

root
 |-- CUSTOMERNO: string (nullable = true)
 |-- tag_cat: string (nullable = true)
 |-- gender_cat: string (nullable = false)
 |-- status_cat: string (nullable = false)
 |-- tenure: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- sanctionamount: double (nullable = true)
 |-- sum_Disubursement_amt: double (nullable = true)
 |-- overdue_interest: double (nullable = true)
 |-- overdue_principal: double (nullable = true)
 |-- bounce_charges: double (nullable = true)
 |-- principal_Outstanding_amt: double (nullable = true)
 |-- no_of_loan_counts: double (nullable = true)
 |-- vintage: double (nullable = true)
 |-- recency: double (nullable = true)
 |-- Age_new: integer (nullable = true)
 |-- MF_Target: integer (nullable = true)



In [79]:
df.columns

['CUSTOMERNO',
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'MF_Target']

In [80]:
dff=df.select(['MF_Target',
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new'
 ])

In [81]:
dff.toPandas().head(2)

,MF_Target,tag_cat,gender_cat,status_cat,tenure,interest_rate,sanctionamount,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Age_new
0,0,online,B,active,6.0,38.0,16000.0,16000.0,0.0,0.0,0.0,6002.0,1.0,0.0,0.0,30
1,0,online,A,active,24.0,18.0,250000.0,500000.0,0.0,0.0,600.0,214282.0,1.0,0.0,0.0,34


In [82]:
dff.groupBy('MF_Target').count().show()

+---------+------+
|MF_Target| count|
+---------+------+
|        0|203291|
|        1| 11663|
+---------+------+



In [83]:
dff.count(),len(dff.columns)

(214954, 16)

In [84]:
dff.columns

['MF_Target',
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new']

In [85]:
dff.toPandas().head()

,MF_Target,tag_cat,gender_cat,status_cat,tenure,interest_rate,sanctionamount,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Age_new
0,0,online,B,active,6.0,38.0,16000.0,16000.0,0.0,0.0,0.0,6002.0,1.0,0.0,0.0,30
1,0,online,A,active,24.0,18.0,250000.0,500000.0,0.0,0.0,600.0,214282.0,1.0,0.0,0.0,34
2,0,online,A,active,6.0,36.0,10000.0,10000.0,355.0,3337.0,0.0,6738.0,1.0,0.0,0.0,35
3,0,online,A,active,18.0,23.0,200000.0,400000.0,0.0,0.0,0.0,171226.0,1.0,0.0,0.0,29
4,0,online,A,active,12.0,29.0,65000.0,260000.0,0.0,0.0,0.0,65000.0,3.0,0.0,0.0,31


# Correlation matrix

In [86]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import pandas as pd

num_col = ['tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new']
assembler = VectorAssembler(inputCols=num_col, outputCol="feature") 
df_vector = assembler.transform(dff).select("feature") 
# ##feature matlab vo num col pura lega and feature name dega

matrix = Correlation.corr(df_vector, "feature").collect()[0][0]
corrmatrix = matrix.toArray().tolist()

In [87]:
pd.DataFrame(corrmatrix, columns=num_col, index=num_col)

,tenure,interest_rate,sanctionamount,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Age_new
tenure,1.000000,-0.334216,0.400616,0.127726,0.078582,-0.046124,0.038909,0.687137,-0.116892,0.027465,0.046291,0.233202
interest_rate,-0.334216,1.000000,-0.164704,-0.066388,0.019229,0.000979,0.027579,-0.251295,0.153935,-0.023843,-0.095229,-0.180875
sanctionamount,0.400616,-0.164704,1.000000,0.435340,0.008470,-0.012431,-0.005139,0.603703,-0.045412,-0.012793,-0.005412,0.109435
sum_Disubursement_amt,0.127726,-0.066388,0.435340,1.000000,0.001000,-0.011680,-0.006384,0.193146,0.008216,0.013811,-0.006693,0.037902
overdue_interest,0.078582,0.019229,0.008470,0.001000,1.000000,0.632996,0.194115,0.014478,0.066957,0.303715,0.239263,0.043420
overdue_principal,-0.046124,0.000979,-0.012431,-0.011680,0.632996,1.000000,0.289649,-0.045159,0.019173,0.366740,0.373304,0.018420
bounce_charges,0.038909,0.027579,-0.005139,-0.006384,0.194115,0.289649,1.000000,-0.015873,-0.013370,0.456859,0.457806,0.056086
principal_Outstanding_amt,0.687137,-0.251295,0.603703,0.193146,0.014478,-0.045159,-0.015873,1.000000,-0.076382,-0.056437,-0.044482,0.179856
no_of_loan_counts,-0.116892,0.153935,-0.045412,0.008216,0.066957,0.019173,-0.013370,-0.076382,1.000000,0.311814,-0.020303,-0.089371
vintage,0.027465,-0.023843,-0.012793,0.013811,0.303715,0.366740,0.456859,-0.056437,0.311814,1.000000,0.811388,0.073876


In [88]:
dff.columns

['MF_Target',
 'tag_cat',
 'gender_cat',
 'status_cat',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new']

# Model Building

# String indexer & OHE

In [89]:
# There are three steps to implement the StringIndexer
# Build the StringIndexer model: specify the input column and output column names.
# Learn the StringIndexer model: fit the model with your data.
# Execute the indexing: call the transform function to execute the indexing process.

In [90]:
stringIndex = StringIndexer(inputCols=['tag_cat', 'gender_cat','status_cat'], 
                       outputCols=['tag_cat_SI', 'gender_cat_SI','status_cat_SI'])

stringIndex_model = stringIndex.fit(dff)

df1_ = stringIndex_model.transform(dff).drop('tag_cat', 'gender_cat','status_cat')

In [91]:
df1_.toPandas().head(2)

,MF_Target,tenure,interest_rate,sanctionamount,sum_Disubursement_amt,overdue_interest,overdue_principal,bounce_charges,principal_Outstanding_amt,no_of_loan_counts,vintage,recency,Age_new,tag_cat_SI,gender_cat_SI,status_cat_SI
0,0,6.0,38.0,16000.0,16000.0,0.0,0.0,0.0,6002.0,1.0,0.0,0.0,30,0.0,1.0,0.0
1,0,24.0,18.0,250000.0,500000.0,0.0,0.0,600.0,214282.0,1.0,0.0,0.0,34,0.0,0.0,0.0


In [92]:
df1_.head(1)

[Row(MF_Target=0, tenure=6.0, interest_rate=38.0, sanctionamount=16000.0, sum_Disubursement_amt=16000.0, overdue_interest=0.0, overdue_principal=0.0, bounce_charges=0.0, principal_Outstanding_amt=6002.0, no_of_loan_counts=1.0, vintage=0.0, recency=0.0, Age_new=30, tag_cat_SI=0.0, gender_cat_SI=1.0, status_cat_SI=0.0)]

In [93]:
df1_.columns

['MF_Target',
 'tenure',
 'interest_rate',
 'sanctionamount',
 'sum_Disubursement_amt',
 'overdue_interest',
 'overdue_principal',
 'bounce_charges',
 'principal_Outstanding_amt',
 'no_of_loan_counts',
 'vintage',
 'recency',
 'Age_new',
 'tag_cat_SI',
 'gender_cat_SI',
 'status_cat_SI']

# VectorAssembler

In [94]:
from pyspark.ml.feature import VectorAssembler

In [95]:
vec_asmbl = VectorAssembler(inputCols=df1_.columns[1:], outputCol='features')

df1_ = vec_asmbl.transform(df1_).select('features', 'MF_Target')

In [96]:
df1_.show(4, truncate=False)

+-------------------------------------------------------------------------------+---------+
|features                                                                       |MF_Target|
+-------------------------------------------------------------------------------+---------+
|(15,[0,1,2,3,7,8,11,13],[6.0,38.0,16000.0,16000.0,6002.0,1.0,30.0,1.0])        |0        |
|(15,[0,1,2,3,6,7,8,11],[24.0,18.0,250000.0,500000.0,600.0,214282.0,1.0,34.0])  |0        |
|[6.0,36.0,10000.0,10000.0,355.0,3337.0,0.0,6738.0,1.0,0.0,0.0,35.0,0.0,0.0,0.0]|0        |
|(15,[0,1,2,3,7,8,11],[18.0,23.0,200000.0,400000.0,171226.0,1.0,29.0])          |0        |
+-------------------------------------------------------------------------------+---------+
only showing top 4 rows



# Train Test OOT data

In [97]:
df1_.count()

214954

In [98]:
train_df,test_df, valid_df = df1_.randomSplit([0.7,.15,.15],seed = 123)  
##if i dont put seed then har time sample change hoga

In [99]:
train_df.count(),test_df.count(),valid_df.count()

(150823, 32199, 31932)

In [100]:
train_df.groupBy('MF_Target').count().withColumn('team_percent', (col('count')/train_df.count())*100).show()
test_df.groupBy('MF_Target').count().withColumn('team_percent', (col('count')/test_df.count())*100).show()
valid_df.groupBy('MF_Target').count().withColumn('team_percent', (col('count')/valid_df.count())*100).show()

+---------+------+-----------------+
|MF_Target| count|     team_percent|
+---------+------+-----------------+
|        0|142642|94.57576099136074|
|        1|  8181|5.424239008639266|
+---------+------+-----------------+

+---------+-----+------------------+
|MF_Target|count|      team_percent|
+---------+-----+------------------+
|        0|30473| 94.63958508028199|
|        1| 1726|5.3604149197180035|
+---------+-----+------------------+

+---------+-----+-----------------+
|MF_Target|count|     team_percent|
+---------+-----+-----------------+
|        0|30176|94.50081423023926|
|        1| 1756|5.499185769760741|
+---------+-----+-----------------+



In [101]:
train_df.show(4, truncate=False)

+-----------------------------------------------------------------------+---------+
|features                                                               |MF_Target|
+-----------------------------------------------------------------------+---------+
|(15,[0,1,2,3,4,7,8,11],[6.0,36.0,10000.0,20000.0,1.0,3685.0,1.0,22.0]) |0        |
|(15,[0,1,2,3,4,7,8,11],[12.0,36.0,9028.0,18056.0,0.06,5772.0,1.0,23.0])|0        |
|(15,[0,1,2,3,5,7,8,11],[1.0,26.0,5998.0,5998.0,96.0,96.0,1.0,23.0])    |0        |
|(15,[0,1,2,3,5,7,8,11],[1.0,26.0,6527.0,6527.0,84.0,84.0,1.0,21.0])    |0        |
+-----------------------------------------------------------------------+---------+
only showing top 4 rows



# Random Forest

In [102]:
rf = RandomForestClassifier(featuresCol='features', labelCol='MF_Target',maxDepth = 3, numTrees=20,seed = 123)
model = rf.fit(train_df)

train_pred = model.transform(train_df)
test_pred = model.transform(test_df)
oot_pred = model.transform(valid_df)

# Here featuresCol is the list of features of the Data Frame, here in our case it is the features column. 
# labelCol is the targeted feature which is labelIndex. 
# rf.fit(train) fits the random forest model to our input dataset named train. 
# rfModel.transform(test) transforms the test dataset.
# This will add new columns to the Data Frame such as prediction, rawPrediction, and probability.

In [103]:
train_pred.show(2)
test_pred.show(2)
oot_pred.show(2)

+--------------------+---------+--------------------+--------------------+----------+
|            features|MF_Target|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+--------------------+----------+
|(15,[0,1,2,3,4,7,...|        0|[19.5108372516474...|[0.97554186258237...|       0.0|
|(15,[0,1,2,3,4,7,...|        0|[19.4502486005393...|[0.97251243002696...|       0.0|
+--------------------+---------+--------------------+--------------------+----------+
only showing top 2 rows

+--------------------+---------+--------------------+--------------------+----------+
|            features|MF_Target|       rawPrediction|         probability|prediction|
+--------------------+---------+--------------------+--------------------+----------+
|(15,[0,1,2,3,5,7,...|        0|[19.5276018208957...|[0.97638009104478...|       0.0|
|(15,[0,1,2,3,5,7,...|        0|[19.5276018208957...|[0.97638009104478...|       0.0|
+--------------------+-------

In [104]:
train_pred.select("MF_Target", "prediction").show(10)

+---------+----------+
|MF_Target|prediction|
+---------+----------+
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
+---------+----------+
only showing top 10 rows



In [105]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='MF_Target', metricName='accuracy')
evaluator.evaluate(train_pred)

0.9458040219329943

In [106]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(train_pred,{metrc.metricName : 'areaUnderROC'})

0.9130469089308786

In [107]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(train_pred)

0.9130469089308786

In [108]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(test_pred)

0.9228606776456442

In [109]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(oot_pred)

0.9172885057623976

In [110]:
model.featureImportances

SparseVector(15, {0: 0.2078, 1: 0.1659, 2: 0.0565, 3: 0.024, 4: 0.1015, 5: 0.0784, 7: 0.038, 9: 0.002, 10: 0.0018, 12: 0.3239})

In [111]:
# 'tenure',
#  'interest_rate',
#  'sanctionamount',
#  'sum_Disubursement_amt',
#  'overdue_interest',
#  'overdue_principal',
#  'bounce_charges',
#  'principal_Outstanding_amt',
#  'no_of_loan_counts',
#  'vintage',
#  'recency',
#  'Age_new',
#  'tag_cat_SI',
#  'gender_cat_SI',
#  'status_cat_SI'

In [112]:
train_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+------+----+
|prediction|     0|   1|
+----------+------+----+
|       0.0|142642|8174|
|       1.0|  NULL|   7|
+----------+------+----+



In [113]:
test_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+-----+----+
|prediction|    0|   1|
+----------+-----+----+
|       0.0|30473|1724|
|       1.0| NULL|   2|
+----------+-----+----+



In [114]:
oot_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+-----+----+
|prediction|    0|   1|
+----------+-----+----+
|       0.0|30176|1755|
|       1.0| NULL|   1|
+----------+-----+----+



# GBT classifier

In [140]:
gb = GBTClassifier(labelCol='MF_Target', maxIter=75, maxDepth=3,stepSize = 0.6)

model = gb.fit(train_df)

train_pred = model.transform(train_df)
test_pred = model.transform(test_df)
oot_pred = model.transform(valid_df)


In [141]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol='MF_Target', metricName='accuracy')
evaluator.evaluate(train_pred)

0.9729550532743679

In [142]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(train_pred,{metrc.metricName : 'areaUnderROC'})

0.9523711659765719

In [143]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(train_pred)

0.9523705104238525

In [144]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(test_pred)

0.9551413292598481

In [145]:
metrc = BinaryClassificationEvaluator(labelCol='MF_Target')
metrc.evaluate(oot_pred)

0.9526218583701517

In [146]:
model.featureImportances

SparseVector(15, {0: 0.235, 1: 0.2204, 2: 0.0355, 3: 0.0175, 4: 0.0867, 5: 0.0137, 6: 0.0026, 7: 0.0356, 8: 0.0025, 9: 0.022, 10: 0.0012, 11: 0.0163, 12: 0.3105, 13: 0.0005, 14: 0.0001})

In [147]:
# 'tenure',
#  'interest_rate',
#  'sanctionamount',
#  'sum_Disubursement_amt',
#  'overdue_interest',
#  'overdue_principal',
#  'bounce_charges',
#  'principal_Outstanding_amt',
#  'no_of_loan_counts',
#  'vintage',
#  'recency',
#  'Age_new',
#  'tag_cat_SI',
#  'gender_cat_SI',
#  'status_cat_SI'

In [148]:
train_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+------+----+
|prediction|     0|   1|
+----------+------+----+
|       0.0|141708|3145|
|       1.0|   934|5036|
+----------+------+----+



In [149]:
test_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+-----+----+
|prediction|    0|   1|
+----------+-----+----+
|       0.0|30285| 656|
|       1.0|  188|1070|
+----------+-----+----+



In [150]:
oot_pred.groupBy('prediction').pivot('MF_Target').count().show()

+----------+-----+----+
|prediction|    0|   1|
+----------+-----+----+
|       0.0|29982| 667|
|       1.0|  194|1089|
+----------+-----+----+



In [151]:
'''
Recall = TP/ (TP+FN)
Precision = TP/ (TP+FP)
'''

'\nRecall = TP/ (TP+FN)\nPrecision = TP/ (TP+FP)\n'

In [156]:
RTRAIN = 5036/ (5036+3145)
PTRAIN = 5036/ (5036+934)

In [157]:
RTRAIN

0.6155726683779489

In [158]:
PTRAIN

0.8435510887772194

In [159]:
RTEST = 1070/ (1070+656)
PTEST = 1070/ (1070+188)

In [160]:
RTEST

0.6199304750869061

In [161]:
PTEST

0.8505564387917329

In [162]:
ROOT = 1089/ (1089+667)
POOT = 1089/ (1089+194)

In [163]:
ROOT

0.6201594533029613

In [164]:
POOT

0.8487918939984411